In [20]:
import requests
import pandas as pd
from pathlib import Path
import time
from IPython.display import clear_output

In [41]:
df = pd.read_csv('EU_result.csv')
matchId = df.loc[len(df)-1]['matchId']
matchId = int(matchId[5:])
print('Last matchId in csv:',matchId)
# matchId = matchId-1
matchId = 6349519768
print('First matchId in new scan:', matchId)

Last matchId in csv: 6349509069
First matchId in new scan: 6349519768


C:\Users\tygan\AppData\Local\Temp\ipykernel_14052\2102881277.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('EU_result.csv')


In [42]:
key = 'RGAPI-331d979e-3c9d-41cb-bc17-07f1d78ec2ef'
url = 'https://europe.api.riotgames.com/lol/match/v5/matches/'
query = '{}EUW1_{}?api_key={}'.format(url, matchId+1, key)

response = requests.get(query)
json = response.json()
display(json['info']['teams'][0]['win'])

True

In [43]:
countAdded = 0
countTried = 1
startTime = round(time.time())

while(countAdded < 50000):
  query = '{}EUW1_{}?api_key={}'.format(url, matchId, key)
  response = requests.get(query)
  json = response.json()

  if response.status_code == 200:
    if json['info']['gameMode'] != 'ARAM':

      participants = json['info']['participants'][::]
      # gameId = json['metadata']['matchId']

      champIds = []
      champNames = []
      index = 1
      for p in participants:
        
        champIds.append(p['championId'])
        champNames.append(p['championName'])

      champs = {'matchId': json['metadata']['matchId'],
                'blue1':champIds[0],
                'blue2':champIds[1],
                'blue3':champIds[2],
                'blue4':champIds[3],
                'blue5':champIds[4],
                'red1':champIds[5],
                'red2':champIds[6],
                'red3':champIds[7],
                'red4':champIds[8],
                'red5':champIds[9],
                'blueWin':json['info']['teams'][0]['win'] 
      }
      
      
      df.loc[len(df)] = champs
      print('matchId {} added, {} total matches added.'.format(json['metadata']['matchId'], countAdded))
      countAdded += 1
  matchId -= 1
  countTried += 1
  if countTried == 80:
    timeElasped = round(time.time()) - startTime
    timeToWait = 120 - timeElasped
    if timeToWait < 0:
        timeToWait = 0
    print('Waiting for', timeToWait, 'seconds to start again')
    df.to_csv('EU_result.csv')
    time.sleep(timeToWait+2)
    print('Restarting queries')
    startTime = round(time.time())
    countTried = 1
    clear_output(wait=True)
# df_champions.drop([0])
# display(df_champions)

matchId EUW1_6349489352 added, 11072 total matches added.
matchId EUW1_6349489346 added, 11073 total matches added.


ReadTimeout: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Read timed out. (read timeout=None)